In [1]:
pip install opencv-python numpy

Note: you may need to restart the kernel to use updated packages.


## Objective
#### The goal of this implementation is to track the fingertip using a webcam by applying color segmentation techniques. This allows us to interact with the system using just our fingers, enabling natural gesture-based interfaces, especially useful in Augmented Reality (AR) applications.

### Concept Behind It
##### To track a fingertip, we follow these steps:

##### Capture a video stream using a webcam.

##### Define a Region of Interest (ROI) where the user is expected to place their hand.

##### Convert the ROI to HSV color space to easily filter out skin tones.

##### Apply a color mask to isolate skin-colored pixels.

##### Detect contours in the skin mask – the largest one is assumed to be the hand.

##### Find the topmost point of the hand contour – this is usually the fingertip.

##### Mark and display the fingertip position on the screen.

In [2]:
import cv2
import numpy as np

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # Mirror the frame
    roi = frame[100:400, 100:400]  # Define Region of Interest (ROI)

    # Convert ROI to HSV
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    # Define skin color range (can be tuned)
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)

    # Threshold the HSV image to get only skin colors
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Apply some filters
    mask = cv2.GaussianBlur(mask, (5, 5), 0)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Get largest contour assumed to be the hand
        cnt = max(contours, key=lambda x: cv2.contourArea(x))

        # Draw the contour
        cv2.drawContours(roi, [cnt], -1, (0, 255, 0), 2)

        # Find the topmost point (assumed fingertip)
        topmost = tuple(cnt[cnt[:, :, 1].argmin()][0])
        cv2.circle(roi, topmost, 8, (255, 0, 0), -1)  # Fingertip mark

    # Draw the ROI rectangle
    cv2.rectangle(frame, (100, 100), (400, 400), (255, 0, 0), 2)
    cv2.imshow("Fingertip Tracker", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC to exit
        break

cap.release()
cv2.destroyAllWindows()


### Why HSV and Not RGB?
#### HSV (Hue, Saturation, Value) separates color information (hue) from lighting (value).

#### This makes skin detection more robust to lighting changes compared to RGB.

## 📸 Region of Interest (ROI)
#### We don’t need to process the entire frame. Instead, we define a small area where the user will place their hand. This makes the system:

##### Faster

##### Less error-prone

##### Easier to test

#### python
##### roi = frame[100:400, 100:400]
#### ✋ Fingertip Detection Logic
##### Once the hand contour is detected, we use:
##### topmost = tuple(cnt[cnt[:, :, 1].argmin()][0])
###### This finds the topmost point in the Y-axis — which is usually a raised finger.

##### A circle is drawn at this point to visually show tracking.